# SECTION-1

CODE TO DOWNLOAD AND EXTRACT IMAGE FILES - NO NEED TO RUN IF images FOLDER IS ALREADY PRESENT IN THE CURRENT WORKING DIRECTORY

In [0]:
!wget --header="Host: doc-0o-2s-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" "https://doc-0o-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/l0nb1suhn7nk2v389qav1jii1uj5maul/1551168000000/14696292948499217652/*/1TaIsOusinIiDnuUcUBsrA4SArO1WAVv0?e=download" -O "Copy of FlipkartGridStage2DataSetImages.zip" -c

In [0]:
!unzip -qq 'Copy of FlipkartGridStage2DataSetImages.zip'

In [0]:
import os as os
len(os.listdir('images/'))

# SECTION-2

Code to read the training csv file of both round-2 and round-3, shuffle it and re-saving it as new_training_combined.csv. 

We also move the training images in a new directory named 'train'

We read test.csv file and move the test images to a new directory 'test'

In [63]:
import pandas as pd
from sklearn.utils import shuffle

train = pd.read_csv('training_set.csv')
trainx = pd.read_csv('training.csv')
trainx = trainx[['image_name', 'x1', 'x2', 'y1', 'y2']]
train = pd.concat([train, trainx], axis = 0)
train = train.drop_duplicates(subset = 'image_name')
train = shuffle(train)
train.to_csv('new_training_combined.csv', index = False)

train = pd.read_csv('new_training_combined.csv')
train.shape

(30970, 5)

In [0]:
train2 = pd.DataFrame([], columns = ['image_name', 'cx', 'cy', 'lx', 'ly'])
train2['image_name'] = train['image_name']
train2['cx'] = (train['x1'] + train['x2'])/2.
train2['cy'] = (train['y1'] + train['y2'])/2.
train2['lx'] = (train['x2'] - train['x1'])
train2['ly'] = (train['y2'] - train['y1'])

#scaler = MinMaxScaler()
#train2[['cx', 'cy', 'lx', 'ly']] = scaler.fit_transform(train2[['cx', 'cy', 'lx', 'ly']])
train2.head()

In [0]:
train2.shape

In [0]:
import pandas as pd
test = pd.read_csv('test.csv')
test.head()

In [0]:
test.shape

In [0]:
import os as os
len(os.listdir('images/'))

In [0]:
!mkdir 'train'

In [0]:
!mkdir 'test'

In [0]:
from shutil import copy
from tqdm import tqdm
for i in tqdm(list(train2['image_name'])):
  copy('images/'+i, 'train/'+i)  

In [0]:
len(os.listdir('train'))

In [0]:
from shutil import move, copy
from tqdm import tqdm
for i in tqdm(list(test['image_name'])):
  copy('images/'+i, 'test/'+i)

In [0]:
len(os.listdir('test'))

In [0]:
from PIL import Image
Image.open('train/'+os.listdir('train')[0])

In [0]:
Image.open('train/'+os.listdir('train')[1])

# SECTION-3 (OPTIONAL TO RUN)

The section has a code to link google colab notebook to google drive

**CODE TO SAVE FILES IN DRIVE**

In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

# SECTION-4

The section contains our model

Run all cells in section to train the model for 160 epochs

**DENSENET201 WITHOUT IMAGENET **

In [0]:
from PIL import Image
from keras.applications.densenet import DenseNet201, preprocess_input
from keras.models import Model
from keras.layers import Dense, Dropout, Reshape, Conv2D
import pandas as pd
import numpy as np
import os as os

IMAGE_SIZE = 130

In [0]:
train = pd.read_csv('new_training_combined.csv')
train.head()

In [0]:
train['x1'] = train['x1']*IMAGE_SIZE/640
train['x2'] = train['x2']*IMAGE_SIZE/640
train['y1'] = train['y1']*IMAGE_SIZE/480
train['y2'] = train['y2']*IMAGE_SIZE/480

In [0]:
train.head()

In [0]:
train2 = pd.DataFrame([], columns = ['image_name', 'x1', 'y1', 'lx', 'ly'])
train2['image_name'] = train['image_name']
train2['x1'] = train['x1']
train2['y1'] = train['y1']
train2['lx'] = (train['x2'] - train['x1'])
train2['ly'] = (train['y2'] - train['y1'])

In [0]:
train2.head()

In [0]:
import numpy as np
from tqdm import tqdm
from PIL import Image

X = np.zeros((len(os.listdir('train/')), IMAGE_SIZE, IMAGE_SIZE, 3), dtype = 'uint8')
Y = np.zeros((len(os.listdir('train/')), 4))
count = 0
for i in tqdm(os.listdir('train')):
  img = Image.open('train/'+i).resize((IMAGE_SIZE, IMAGE_SIZE)).convert('RGB')
  X[count] = preprocess_input(np.array(img, dtype = 'uint8'))
  img.close()
  coord = []
  coord.append(float(train2.loc[train2['image_name'] == i]['x1']))
  coord.append(float(train2.loc[train2['image_name'] == i]['y1']))
  coord.append(float(train2.loc[train2['image_name'] == i]['lx']))
  coord.append(float(train2.loc[train2['image_name'] == i]['ly']))
  #coord.append(int(train.loc[train['image_name'] == i]['y1']))
  #coord.append(int(train.loc[train['image_name'] == i]['y2']))
  Y[count] = np.array(coord)
  count += 1

In [0]:
X.shape

In [0]:
Y.shape

In [0]:
Image.fromarray(X[10], 'RGB')

In [0]:
#from sklearn.model_selection import train_test_split

#X_train, Y_train, X_test, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 42)

#print(X_train.shape)
#print(X_test.shape)
#print(Y_train.shape)
#print(Y_test.shape)

#X = None
#Y = None

In [0]:
from keras.applications.densenet import DenseNet201
def create_model():
  ALPHA = 1.0
  model = DenseNet201(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights = None) #weights = None ensures random initialization of weights 
  x = model.layers[-1].output
  x = Conv2D(4, kernel_size=3, strides = 2, name="coords")(x)
  x = Reshape((4,))(x)
  return Model(inputs=model.input, outputs=x)

In [0]:
def iou(y_true, y_pred):
  x11 = y_true[:,0]
  x12 = tf.add(y_true[:,0], y_true[:,2])
  y11 = y_true[:,1]
  y12 = tf.add(y_true[:,1], y_true[:,3])
  
  
  x21 = y_pred[:,0]
  x22 = tf.add(y_pred[:,0], y_pred[:,2])
  y21 = y_pred[:,1]
  y22 = tf.add(y_pred[:,1], y_pred[:,3])
  
  xi1 = tf.maximum(x11, x21)
  xi2 = tf.minimum(x12, x22)
  yi1 = tf.maximum(y11, y21)
  yi2 = tf.minimum(y12, y22)
  
  inter_area = (yi2 - yi1)*(xi2 - xi1)
  
  box1_area = (x12 - x11)*(y12 - y11)
  box2_area = (x22 - x21)*(y22 - y21)
  
  union_area = box1_area + box2_area - inter_area
  
  iou = inter_area/union_area
  return iou

In [0]:
IMAGE_SIZE = 130
model = create_model()
model.summary()

In [0]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import Adam

learning_rate = 0.001
opt = Adam(lr = 0.001, amsgrad = True, decay = learning_rate/380)
PATIENCE = 50
model.compile(optimizer = opt, loss = 'mean_squared_error' , metrics = [iou])
#stop = EarlyStopping(monitor="val_iou", patience=PATIENCE, mode="max")

In [0]:
#model.load_weights('drive/densenet201_385_no_val.h5')
model.fit(X, Y, epochs = 380, batch_size = 32, shuffle = True)

In [0]:
model.save_weights('densenet201_380_no_val.h5')
#model.save_weights('drive/densenet201_420_no_val.h5')

# SECTION-5

Section contains code to generate the output csv file

In [0]:
model.load_weights('densenet201_380_no_val.h5')

In [0]:
test = pd.read_csv('test.csv')
test.head()

In [0]:
len(os.listdir('test/'))

In [0]:
import numpy as np
from tqdm import tqdm
from PIL import Image

test_X = np.zeros((len(os.listdir('test/')), IMAGE_SIZE, IMAGE_SIZE, 3), dtype = 'uint8')
sub = []
count = 0
for i in tqdm(os.listdir('test')):
  img = Image.open('test/'+i).resize((IMAGE_SIZE, IMAGE_SIZE)).convert('RGB')
  test_X[count] = preprocess_input(np.array(img, dtype = 'uint8'))
  img.close()
  sub.append(i)
  count += 1
sub = np.array(sub)

In [0]:
test_X.shape

In [0]:
sub.shape

In [0]:
pred = model.predict(test_X, verbose = 1)

In [0]:
sub = np.array(sub, copy = False)
#sub = sub.reshape((24045,1))
sub = np.expand_dims(sub, axis = 1)
pred = np.append(sub, pred, axis = 1)
pred.shape

In [0]:
df = pd.DataFrame(pred, columns = ['image_name', 'x', 'y', 'lx', 'ly'])
df

In [0]:
df["x"] = pd.to_numeric(df["x"],errors='coerce')
df["y"] = pd.to_numeric(df["y"],errors='coerce')
df["lx"] = pd.to_numeric(df["lx"],errors='coerce')
df["ly"] = pd.to_numeric(df["ly"],errors='coerce')

In [0]:
x = df['x'].values
y = df['y'].values
lx = df['lx'].values
ly = df['ly'].values

y

In [0]:
sub_df = pd.DataFrame([], columns = ['image_name', 'x1', 'x2', 'y1', 'y2'])
sub_df.image_name = df.image_name
sub_df.x1 = np.multiply(x, 640/IMAGE_SIZE)
sub_df.x2 = np.multiply(x + lx, 640/IMAGE_SIZE)
sub_df.y1 = np.multiply(y, 480/IMAGE_SIZE)
sub_df.y2 = np.multiply(y + ly, 480/IMAGE_SIZE)
sub_df

In [0]:
sub_df.to_csv('submission_31.csv', index = False)